In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 118.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

# Twitter 2015

#### Twitter fake news dataset

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/Twitter2015/tweetstrain2015.csv')
train.head()

,Unnamed: 0.1,Unnamed: 0,tweetId,tweetText,label
0,0,0,263046056240115712,se acuerdan de la pel cula el despu de ana rec...,fake
1,1,1,262995061304852481,milenagimon miren sandi en ny tremenda imagen ...,fake
2,2,2,262979898002534400,buena la foto del hurac n sandi recuerda la pe...,fake
3,3,3,262996108400271360,scari shit hurrican ny http co e jlbufh,fake
4,4,4,263018881839411200,fave place world nyc hurrican sandi statueofli...,fake


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/Twitter2015/tweetstest2015.csv')
test.head()

,Unnamed: 0.1,Unnamed: 0,tweetId,tweetText,label
0,0,0,578854927457349632,kereeen rt shyman eclips iss http co je hcfpvfn,fake
1,1,1,578874632670953472,absolut beauti rt shyman eclips iss http co oq...,fake
2,2,2,578891261353984000,shyman eclips iss http co c vfboscrj wow amaz,fake
3,3,3,578846612312748032,eclips iss http co en otvsu,fake
4,4,4,578975333841551360,ebonfigli clips vue de l iss autr chose http c...,fake


In [ ]:
#encode output
label_mapping = {'fake': 1, 'real': 0}
train.label = train.label.map(label_mapping)
test.label = test.label.map(label_mapping)

### Train val split

In [ ]:
train = train.dropna()
X_train, X_val, y_train, y_val = train_test_split(train['tweetText'], train['label'], test_size=0.2, random_state=42)
X_test, y_test = test['tweetText'], test.label

#### Preprocessing the dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_train_encoded = tokenizer(
    list(X_train.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_val_encoded = tokenizer(
    list(X_val.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_test_encoded = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = FakeNewsDataset(X_train_encoded, y_train.values)
val_dataset = FakeNewsDataset(X_val_encoded, y_val.values)
test_dataset = FakeNewsDataset(X_test_encoded, y_test.values)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Training

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
import evaluate
import numpy as np
import os
os.environ['HF_MLFLOW_LOG_ARTIFACTS'] = "1" # save models as artifact for the expirment

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels) 

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP Project/saved_models/bert/',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,
    evaluation_strategy='steps',
    eval_steps=400,
    load_best_model_at_end=True,
    save_total_limit=3,
    save_steps=400

)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
400,0.552800,0.386814,0.847689,0.888261,0.847965,0.932578
800,0.422100,0.442426,0.855742,0.892428,0.864879,0.921791
1200,0.332000,0.411481,0.879902,0.912299,0.867282,0.962244
1600,0.297600,0.409402,0.884104,0.911710,0.901847,0.921791
2000,0.276800,0.305877,0.904762,0.929351,0.896293,0.964941
2400,0.277600,0.365511,0.906513,0.930451,0.899748,0.963323
2800,0.295100,0.317442,0.902661,0.926103,0.912998,0.939590
3200,0.210400,0.401569,0.912115,0.934618,0.903778,0.967638
3600,0.208300,0.378559,0.914216,0.935847,0.909415,0.963862
4000,0.231800,0.343998,0.908263,0.930245,0.918507,0.942287


<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=7140, training_loss=0.236467664382037, metrics={'train_runtime': 7008.2415, 'train_samples_per_second': 8.148, 'train_steps_per_second': 1.019, 'total_flos': 1.50249568163328e+16, 'train_loss': 0.236467664382037, 'epoch': 5.0})

### Calculate performance

In [ ]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/saved_models/bert/checkpoint-6800/')

In [ ]:
print(f"Accuracy: {acc / len(test_loader)}")

In [ ]:
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

tp = 0.0  # true positives
fp = 0.0  # false positives
tn = 0.0  # true negatives
fn = 0.0  # false negatives

with torch.no_grad():
    for data in tqdm(test_loader):
        input_ids, labels = data['input_ids'].to(device), data['labels'].to(device)
        out = torch.softmax(model(input_ids).logits, dim=1)
        preds = torch.argmax(out, dim=1)
        
        # update confusion matrix
        tp += ((preds == 1) & (labels == 1)).sum().item()
        fp += ((preds == 1) & (labels == 0)).sum().item()
        tn += ((preds == 0) & (labels == 0)).sum().item()
        fn += ((preds == 0) & (labels == 1)).sum().item()
        
        acc = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_score = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


  0%|          | 0/118 [00:00<?, ?it/s]<ipython-input-10-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 118/118 [01:41<00:00,  1.17it/s]

Accuracy: 0.9025
Precision: 0.9202
Recall: 0.9375
F1 Score: 0.9288


In [ ]:
eval_result = trainer.evaluate(eval_dataset=val_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-10-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 0.6553
eval_accuracy: 0.6492
eval_f1: 0.7873
eval_precision: 0.6492
eval_recall: 1.0000
eval_runtime: 88.8387
eval_samples_per_second: 32.1480
eval_steps_per_second: 4.0190


In [ ]:
eval_result = trainer.evaluate(eval_dataset=test_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-10-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 0.6379
eval_accuracy: 0.6780
eval_f1: 0.8081
eval_precision: 0.6780
eval_recall: 1.0000
eval_runtime: 111.1434
eval_samples_per_second: 33.7850
eval_steps_per_second: 4.2290


# Twitter 2016

#### Twitter fake news dataset

In [23]:
train = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/Twitter2016/tweetstrain2016.csv')
train.head()

,Unnamed: 0.1,Unnamed: 0,tweetId,tweetText,label
0,0,0,324597532548276224,need fed solv bostonbomb chan http co exqtpzqqbg,fake
1,1,1,325145334739267584,pic comparison boston suspect sunil tripathi f...,fake
2,2,2,325152091423248385,complet convinc sunil tripathi fellow http co ...,fake
3,3,3,324554646976868352,brutal lo que se pued conseguir en colaboraci ...,fake
4,4,4,324315545572896768,chan bomb throw http co diyso lxqm http co nxb...,fake


In [24]:
test = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/Twitter2016/tweetstest2016.csv')
test.head()

,Unnamed: 0.1,Unnamed: 0,tweetId,tweetText,label
0,0,0,651118294447951872,antiterror arabianblood russianfeder syria dai...,fake
1,1,1,651115824065830912,n http co ws jxmgj,fake
2,2,2,651095856662360065,http co lz awn nhttp co gvsduzmaxa,fake
3,3,3,651086828234104832,http co kg quidotf http co dkfmxppje,fake
4,4,4,651034616007106560,nthe airstrik isi ammunit depot near talbiseh ...,fake


In [25]:
#encode output
label_mapping = {'fake': 1, 'real': 0}
train.label = train.label.map(label_mapping)
test.label = test.label.map(label_mapping)

### Train val split

In [26]:
train = train.dropna()
X_train, X_val, y_train, y_val = train_test_split(train['tweetText'], train['label'], test_size=0.2, random_state=42)
X_test, y_test = test['tweetText'], test.label

#### Preprocessing the dataset

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [28]:
X_train_encoded = tokenizer(
    list(X_train.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_val_encoded = tokenizer(
    list(X_val.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_test_encoded = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [29]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [30]:
train_dataset = FakeNewsDataset(X_train_encoded, y_train.values)
val_dataset = FakeNewsDataset(X_val_encoded, y_val.values)
test_dataset = FakeNewsDataset(X_test_encoded, y_test.values)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Training

In [31]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
import evaluate
import numpy as np
import os
os.environ['HF_MLFLOW_LOG_ARTIFACTS'] = "1" # save models as artifact for the expirment

In [33]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels) 

In [34]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP Project/saved_models/bert/',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,
    evaluation_strategy='steps',
    eval_steps=400,
    load_best_model_at_end=True,
    save_total_limit=3,
    save_steps=400

)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [35]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
400,0.508500,0.769098,0.768714,0.776369,0.940780,0.660874
800,0.377100,0.358047,0.876200,0.903032,0.861377,0.948920
1200,0.347000,0.341580,0.875560,0.905697,0.839173,0.983676
1600,0.313100,0.303967,0.890595,0.913897,0.875543,0.955766
2000,0.274300,0.317967,0.897313,0.918919,0.883010,0.957873
2400,0.253400,0.293394,0.893794,0.915608,0.885012,0.948394
2800,0.257100,0.333678,0.893474,0.917635,0.865205,0.976830
3200,0.268900,0.420149,0.902111,0.921979,0.893722,0.952080
3600,0.198200,0.388346,0.903071,0.922763,0.894269,0.953133
4000,0.210400,0.390685,0.901472,0.918734,0.920677,0.916798


<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=7815, training_loss=0.22438241540050935, metrics={'train_runtime': 1295.056, 'train_samples_per_second': 48.272, 'train_steps_per_second': 6.034, 'total_flos': 1606287854085000.0, 'train_loss': 0.22438241540050935, 'epoch': 5.0})

### Calculate performance

In [37]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/saved_models/bert/checkpoint-7600/')

In [38]:
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

tp = 0.0  # true positives
fp = 0.0  # false positives
tn = 0.0  # true negatives
fn = 0.0  # false negatives

with torch.no_grad():
    for data in tqdm(test_loader):
        input_ids, labels = data['input_ids'].to(device), data['labels'].to(device)
        out = torch.softmax(model(input_ids).logits, dim=1)
        preds = torch.argmax(out, dim=1)
        
        # update confusion matrix
        tp += ((preds == 1) & (labels == 1)).sum().item()
        fp += ((preds == 1) & (labels == 0)).sum().item()
        tn += ((preds == 0) & (labels == 0)).sum().item()
        fn += ((preds == 0) & (labels == 1)).sum().item()
        
        acc = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_score = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


  0%|          | 0/69 [00:00<?, ?it/s]<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 69/69 [01:13<00:00,  1.07s/it]

Accuracy: 0.5443
Precision: 0.7235
Recall: 0.2648
F1 Score: 0.3877


In [39]:
eval_result = trainer.evaluate(eval_dataset=val_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 0.2934
eval_accuracy: 0.8938
eval_f1: 0.9156
eval_precision: 0.8850
eval_recall: 0.9484
eval_runtime: 13.7372
eval_samples_per_second: 227.5580
eval_steps_per_second: 28.4630
epoch: 5.0000


In [40]:
eval_result = trainer.evaluate(eval_dataset=test_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-29-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 1.1548
eval_accuracy: 0.6174
eval_f1: 0.6815
eval_precision: 0.6235
eval_recall: 0.7513
eval_runtime: 77.5944
eval_samples_per_second: 28.0560
eval_steps_per_second: 3.5180
epoch: 5.0000
